<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Check-in" data-toc-modified-id="Final-Project-Check-in-1">Final Project Check-in</a></span></li><li><span><a href="#Group-Name" data-toc-modified-id="Group-Name-2">Group Name</a></span></li><li><span><a href="#Student-Names" data-toc-modified-id="Student-Names-3">Student Names</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Load Data</a></span></li><li><span><a href="#Fit-scikit-learn-model" data-toc-modified-id="Fit-scikit-learn-model-5">Fit scikit-learn model</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-6">Evaluation Metric</a></span></li></ul></div>

Final Project Check-in
------

Group Name
-----

The Grace Hoppers 

Student Names
----

1. Akansha Shrivastava
2. Ivette Sulca
3. Bing Wang

Load Data
-----

In [1]:
from collections import Counter
import imblearn
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import geonamescache  #sudo pip install geonamescache

In [2]:
data_pd = pd.read_csv('../../data/epi_r.csv')
data_json = pd.read_json('../../data/full_format_recipes.json',)
#data_pd = pd.read_csv('data/epi_r.csv')
#data_json = pd.read_json('data/full_format_recipes.json',)

In [3]:
# Deleting null predictive variables from the dataset
data_pd = data_pd.loc[(data_pd.rating>0) & (~data_pd.rating.isna())]

# Deleting drinks
data_pd = data_pd.loc[(data_pd.drink==0) & (data_pd.drinks==0) & (data_pd.cocktail==0)]

# Deleting repeated titles
data_pd.drop_duplicates(subset=['title'], inplace=True)  

In [4]:
# Remove target, "rating", from data_pd
y = data_pd.iloc[:,1]
data_pd = pd.concat([data_pd.iloc[:,0], data_pd.iloc[:,2:]], axis=1)

In [5]:
# Create feature: holidays: Dummy for world holiday 
# (a celebratory day attached to date in calendar-- birthday, graduation, anniversary, etc. not included)
holidays = ["bastille day", "christmas", "christmas eve", "cinco de mayo", "columbus", 
 "diwali", "easter", "father's day", "fourth of july", "friendsgiving", "halloween",
 "hanukkah", "kwanzaa", "labor day", "lunar new year", "mother's day", "new year's day",
 "new year's eve", "oktoberfest", "passover", "persian new year", "purim", "ramadan", 
 "rosh hashanah/yom kippur", "st. patrick's day", "sukkot", "thanksgiving", 
 "valentine's day"]

data_pd["holiday"] = np.where(data_pd[holidays].sum(axis=1)>0, 1, 0)

In [6]:
#FAT: In grams but it can mislead depending of the number of portions (Paella for example)

#Delete bigger portions(Paella) and outliers
y = y.loc[((data_pd.fat>=0) & (data_pd.fat<=200)) | (data_pd.fat.isna())]
data_pd = data_pd.loc[((data_pd.fat>=0) & (data_pd.fat<=200)) | (data_pd.fat.isna())]

#Median imputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
fat_clean = imp.fit_transform(data_pd.fat.values.reshape(-1,1))
fat_clean = pd.DataFrame(data=fat_clean, columns=['fat_clean'])
data_pd['fat'] = fat_clean.fat_clean.values

In [7]:
#PROTEIN CLEANING

#Again, considering values lower than 200: deleting 62 rows...
y = y.loc[((data_pd.protein>=0) & (data_pd.protein<=200)) | (data_pd.protein.isna())]
data_pd = data_pd.loc[((data_pd.protein>=0) & (data_pd.protein<=200)) | (data_pd.protein.isna())]

#Median imputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
protein_clean = imp.fit_transform(data_pd.protein.values.reshape(-1,1))
protein_clean = pd.DataFrame(data=protein_clean ,columns=['protein_clean'])
data_pd['protein']=protein_clean.protein_clean.values


In [8]:
# SODIUM

#Unit: miligrams
#Very different values, so we will impute only:

imp = SimpleImputer(missing_values=np.nan, strategy='median')
sodium_clean = imp.fit_transform(data_pd.sodium.values.reshape(-1,1))
sodium_clean = pd.DataFrame(data=sodium_clean ,columns=['sodium_clean'])
data_pd['sodium']=sodium_clean.sodium_clean.values

In [9]:
# filtering columns related to "healthy"

# selecting all the relevant columns

selected = ["fat free", "healthy", "low cal", "quick and healthy", 
"low carb",
"low cholesterol",
"low fat",
"low sodium",
"low sugar",
"low/no sugar"]

In [10]:
# filtering rows which have "selected" columns as 1 -> healthy
data_pd["allhealthy"] = 0
for col in selected:
    data_pd.loc[data_pd[col] == 1, "allhealthy"] = 1

In [11]:
# replacing missing values in calories by mean

median = data_pd["calories"].median()
data_pd.loc[data_pd["calories"].isna(), "calories"] = median

In [12]:
# Adding complexity preparation from JSON file

data_json.drop_duplicates(subset=['title'],inplace=True)  
data_json['directions_n_characters']=data_json['directions'].astype(str).str.len()
data_json['ingredients_quantity']=data_json['ingredients'].str.len()
data_json['directions_n_steps']=data_json['directions'].astype(str).str.replace('[','').str.replace(']','').str.split("',").apply(lambda x: len(x))
data_json.loc[data_json.desc.isna(), "desc_n_characters"] = 0
data_json.loc[data_json.desc.notna(), "desc_n_characters"] = data_json["desc"].astype(str).str.len()

data_json2=data_json[['title','directions_n_characters','ingredients_quantity',
                      'directions_n_steps', 'desc_n_characters']]

data_pd=pd.merge(data_pd, data_json2, on='title', how="left")

#Change by imputer
#Median imputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')

directions_n_characters = imp.fit_transform(data_pd.directions_n_characters.values.reshape(-1,1))
directions_n_characters = pd.DataFrame(data=directions_n_characters ,columns=['directions_n_characters'])
data_pd['directions_n_characters']=directions_n_characters.directions_n_characters.values


ingredients_quantity = imp.fit_transform(data_pd.ingredients_quantity.values.reshape(-1,1))
ingredients_quantity = pd.DataFrame(data=ingredients_quantity ,columns=['ingredients_quantity'])
data_pd['ingredients_quantity']=ingredients_quantity.ingredients_quantity.values


directions_n_steps = imp.fit_transform(data_pd.directions_n_steps.values.reshape(-1,1))
directions_n_steps = pd.DataFrame(data=directions_n_steps ,columns=['directions_n_steps'])
data_pd['directions_n_steps']=directions_n_steps.directions_n_steps.values


desc_n_characters = imp.fit_transform(data_pd.desc_n_characters.values.reshape(-1,1))
desc_n_characters = pd.DataFrame(data=desc_n_characters ,columns=['desc_n_characters'])
data_pd['desc_n_characters']=desc_n_characters.desc_n_characters.values


In [13]:
data_pd.shape

(15437, 685)

In [14]:
# Identifying locations:
column_names=np.array(data_pd.columns)
column_names=[c.strip().upper() for c in column_names]

gc = geonamescache.GeonamesCache()
countries = gc.get_countries_by_names()
cities = gc.get_cities()
states = gc.get_us_states()

dict_countries=dict()
for k,v in countries.items():
    dict_countries[k.upper()]=[v['geonameid'],v['iso'],v['iso3']]

dict_countries2=dict()
for k,v in countries.items():
    dict_countries2[v['iso'].upper()]=k.upper()

dict_cities=dict()
for k,v in cities.items():
    dict_cities[v['name'].upper()]=[v['geonameid'],v['countrycode']]

dict_states=dict()    
for k,v in states.items():
    dict_states[v['name'].upper()] = [v['geonameid'],v['code'], 'US' ]


In [15]:
# Make dummy variable of if there is a location listed or not

#1. Identifying countries, states and cities

locs=[]
for col in data_pd.columns:    
    if col.upper() in dict_countries:
        if col.upper() not in ['TURKEY']:
#            data_pd.loc[data_pd[col]==1,'country_id'] = dict_countries[col.upper()][1]
#            data_pd.loc[data_pd[col]==1,'country_name'] = col.upper()   
            locs.append(col)
 

    if col.upper() in dict_states:        
#        data_pd.loc[data_pd[col]==1,'state_id'] = dict_states[col.upper()][1]
#        data_pd.loc[data_pd[col]==1,'state_name'] = col.upper()   
#        data_pd.loc[data_pd[col]==1,'country_id'] = 'US'
#        data_pd.loc[data_pd[col]==1,'country_name'] = 'UNITED STATES' 
        locs.append(col)
 
    if col.upper() in dict_cities:     
        if col.upper() not in ['SPRING','ORANGE','WALNUT','LEEK','WEDDING','PLUM','TEQUILA','DATE','PAPAYA','MARSALA','SAKE','RYE','GOUDA','HOLIDAY']:
 #           data_pd.loc[data_pd[col]==1,'city_id'] = dict_cities[col.upper()][0]
 #           data_pd.loc[data_pd[col]==1,'city_name'] = col.upper()   
 #           data_pd.loc[data_pd[col]==1,'country_id'] = dict_cities[col.upper()][1]
 #           data_pd.loc[data_pd[col]==1,'country_name'] = dict_countries2[dict_cities[col.upper()][1]] 
            locs.append(col)

In [16]:
data_pd["locations"] = np.where(data_pd[locs].sum(axis=1)>0, 1, 0)

Fit scikit-learn model
----

In [17]:
# Using the below features in the model:
X = data_pd[["calories", "fat", "protein", "sodium", "22-minute meals",
             "3-ingredient recipes", "holiday", "allhealthy",
             "directions_n_characters", "ingredients_quantity",
             "directions_n_steps"]]

In [18]:
# y (rating) only appears as 6 different floats between 1 and 4
# Convert floats to categorical variables, 0 to 6, for classification

y_discrete = pd.cut(y, bins=7, labels=np.arange(7), right=False)
y_discrete.value_counts()

5    6966
4    4479
6    2044
3    1281
2     445
0     123
1      99
Name: rating, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_discrete, test_size=0.20, stratify=y_discrete)

### Creation of pipelines

In [29]:
#Creating pipelines
pipe_lr = Pipeline([('scl', StandardScaler()),
                ('clf', LogisticRegression(random_state=42))])

pipe_kn = Pipeline([('scl', StandardScaler()),
                ('clf', KNeighborsClassifier())])

pipe_nb = Pipeline([
                ('clf', GaussianNB())])

pipe_svc = Pipeline([#('scl', StandardScaler()),
                ('clf', SVC(random_state=42))])

pipe_rc = Pipeline([
                ('clf', RandomForestClassifier(random_state=42))])

pipelines=[pipe_lr,pipe_kn,pipe_nb,pipe_svc,pipe_rc]

In [30]:
# Training the pipelines
for pipe in pipelines:
    pipe.fit(X_train, y_train)

/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.2

In [31]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

for pipe in pipelines:
    y_pred = pipe.predict(X_train)
    
    model_name=pipe.get_params()['clf'].__class__.__name__
    
    print(model_name)
    print(classification_report(y_train, y_pred))
    


LogisticRegression
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        98
           1       0.00      0.00      0.00        79
           2       0.00      0.00      0.00       356
           3       0.00      0.00      0.00      1025
           4       0.38      0.01      0.01      3583
           5       0.45      1.00      0.62      5573
           6       0.25      0.00      0.00      1635

    accuracy                           0.45     12349
   macro avg       0.15      0.14      0.09     12349
weighted avg       0.35      0.45      0.28     12349



/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.31      0.11      0.16        98
           1       0.26      0.08      0.12        79
           2       0.36      0.15      0.22       356
           3       0.42      0.27      0.33      1025
           4       0.50      0.61      0.55      3583
           5       0.61      0.73      0.67      5573
           6       0.51      0.16      0.25      1635

    accuracy                           0.56     12349
   macro avg       0.43      0.30      0.33     12349
weighted avg       0.54      0.56      0.53     12349

GaussianNB
              precision    recall  f1-score   support

           0       0.08      0.01      0.02        98
           1       0.00      0.00      0.00        79
           2       0.00      0.00      0.00       356
           3       0.10      0.48      0.17      1025
           4       0.29      0.01      0.02      3583
           5       0.49      0.62      0.54   

/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVC
              precision    recall  f1-score   support

           0       1.00      0.81      0.89        98
           1       1.00      0.68      0.81        79
           2       1.00      0.79      0.88       356
           3       1.00      0.84      0.92      1025
           4       0.95      0.94      0.95      3583
           5       0.91      0.99      0.95      5573
           6       1.00      0.87      0.93      1635

    accuracy                           0.94     12349
   macro avg       0.98      0.85      0.90     12349
weighted avg       0.94      0.94      0.94     12349

RandomForestClassifier
              precision    recall  f1-score   support

           0       0.98      0.93      0.95        98
           1       1.00      0.96      0.98        79
           2       1.00      0.97      0.98       356
           3       0.99      0.97      0.98      1025
           4       0.97      0.98      0.98      3583
           5       0.97      0.99      0.98      55

### Only SVC

In [27]:
clf = SVC(random_state=42).fit(X_train, y_train)

y_pred = clf.predict(X_train)
cm = confusion_matrix(y_train, y_pred)
print(cm)
print(classification_report(y_train, y_pred))

/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[  79    0    0    0    4   15    0]
 [   0   54    0    0    7   17    1]
 [   0    0  280    0   17   59    0]
 [   0    0    0  865   42  116    2]
 [   0    0    0    0 3380  200    3]
 [   0    0    0    0   43 5529    1]
 [   0    0    0    0   57  162 1416]]
              precision    recall  f1-score   support

           0       1.00      0.81      0.89        98
           1       1.00      0.68      0.81        79
           2       1.00      0.79      0.88       356
           3       1.00      0.84      0.92      1025
           4       0.95      0.94      0.95      3583
           5       0.91      0.99      0.95      5573
           6       1.00      0.87      0.93      1635

    accuracy                           0.94     12349
   macro avg       0.98      0.85      0.90     12349
weighted avg       0.94      0.94      0.94     12349



In [21]:
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[   0    0    0    0    3   22    0]
 [   0    0    0    0    1   19    0]
 [   0    0    0    0    3   86    0]
 [   0    0    1    0   20  234    1]
 [   0    0    0    0   46  846    4]
 [   0    0    2    1   45 1342    3]
 [   0    0    1    0   30  378    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        89
           3       0.00      0.00      0.00       256
           4       0.31      0.05      0.09       896
           5       0.46      0.96      0.62      1393
           6       0.00      0.00      0.00       409

    accuracy                           0.45      3088
   macro avg       0.11      0.14      0.10      3088
weighted avg       0.30      0.45      0.31      3088



/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Evaluation Metric
----

In [22]:
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[   0    0    0    0    3   22    0]
 [   0    0    0    0    1   19    0]
 [   0    0    0    0    3   86    0]
 [   0    0    1    0   20  234    1]
 [   0    0    0    0   46  846    4]
 [   0    0    2    1   45 1342    3]
 [   0    0    1    0   30  378    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        89
           3       0.00      0.00      0.00       256
           4       0.31      0.05      0.09       896
           5       0.46      0.96      0.62      1393
           6       0.00      0.00      0.00       409

    accuracy                           0.45      3088
   macro avg       0.11      0.14      0.10      3088
weighted avg       0.30      0.45      0.31      3088



/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
print("Unweighted F1 score overall: ", round(f1_score(y_test, y_pred, average='macro'), 3))

Unweighted F1 score overall:  0.101


/Users/ivettesulca/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
